In [ ]:
# Author: Matheus Gomes Correia
# License: CC-BY-NC-SA 4.0

# Import libraries

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import entropy as calculate_entropy
from kmodes.kprototypes import KPrototypes
from scipy.stats import chi2_contingency
import seaborn as sns
import matplotlib.pyplot as plt
import osmnx as ox
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.metrics import silhouette_samples
import gc
import ast
import re
import os
import glob

# Define Constants and File Paths

In [ ]:
notebook_dir = os.path.abspath('')
PROJECT_ROOT = os.path.dirname(notebook_dir)

# Define all directories based on the Project Root
DATA_DIR = os.path.join(PROJECT_ROOT, "data") # The folder where city subfolders are located
RESULTS_DIR = os.path.join(PROJECT_ROOT, "results")
FIGURES_DIR = os.path.join(RESULTS_DIR, "figures", "clustering")

os.makedirs(RESULTS_DIR, exist_ok=True) 
os.makedirs(FIGURES_DIR, exist_ok=True)

all_city_gdfs = []

print(f"Project Root: {PROJECT_ROOT}")
print(f"Data will be loaded from: {DATA_DIR}")
print(f"Results will be saved to: {RESULTS_DIR}")
print(f"Figures will be saved to: {FIGURES_DIR}")
print("-" * 20)
print("Searching for processed network files...")

# Find all the individual city network files generated by the previous script
network_files = glob.glob(os.path.join(DATA_DIR, "*", "*_processed_network.gpkg"))

if not network_files:
    raise FileNotFoundError(f"No processed network files (*_processed_network.gpkg) found in the subdirectories of '{DATA_DIR}'. Please run the first script to generate them.")

print(f"Found {len(network_files)} city network files. Loading and combining...")

for filepath in network_files:
    try:
        # --- Extract city_id from the folder path ---
        city_id = os.path.basename(os.path.dirname(filepath))

        gdf_city = gpd.read_file(filepath)
        if gdf_city.empty:
            print(f"  - Skipping empty file: {filepath}")
            continue

        # --- Create the 'unique_edge_id' ---
        # Combining the city_id with the row index from its own file.
        # This creates a unique and traceable ID for every road segment.
        gdf_city = gdf_city.reset_index(drop=True)
        gdf_city['unique_edge_id'] = f"{city_id}_" + gdf_city.index.astype(str)

        # Add city_id as a regular column for easy filtering later
        gdf_city['city_id'] = city_id
        
        all_city_gdfs.append(gdf_city)
        print(f"  - Loaded {city_id} ({len(gdf_city)} edges)")

    except Exception as e:
        print(f"  - FAILED to load or process {filepath}. Error: {e}")

# --- Combine all cities into a single GeoDataFrame ---
if not all_city_gdfs:
    raise ValueError("No city data could be loaded. Cannot proceed.")

df = pd.concat(all_city_gdfs, ignore_index=True)

print("\nFull dataset combined successfully.")
print("Shape:", df.shape)
print("Memory usage (approx):", df.memory_usage(deep=True).sum() / (1024**2), "MB")
print("Columns:", df.columns.tolist())
print("\nData head (note the new 'unique_edge_id' and 'city_id' columns):")
print(df[['unique_edge_id', 'highway', 'length', 'city_id', 'geometry']].head())


# --- FILTERING STEP ---
# Uncomment this whole part to help debugging or only perform clustering with a single city

#city_to_keep = 'fortaleza'
#print(f"\nFiltering for rows where 'city_id' is '{city_to_keep}'...")

#if 'city_id' not in df.columns:
#    raise ValueError("Column 'city_id' not found in the DataFrame!")

#df = df[df['city_id'] == city_to_keep].copy()

#print(f"Filtering complete.")
#print("Filtered dataset shape:", df.shape)

In [ ]:
print("\n--- Top 50 Value Counts for Each Column (Raw Data) ---")
for column in df.columns:
    print(f"\n--- Column: {column} ---")
    # Using dropna=False to see NaNs if they are frequent
    value_counts_series = df[column].value_counts(dropna=False).head(50)
    if value_counts_series.empty:
        print("No values in this column (or all are unique and >50).")
    else:
        # For better readability, especially if values are long strings
        with pd.option_context('display.max_colwidth', None, 'display.width', 1000):
            print(value_counts_series)
print("-" * 50)

# Cleaning data

In [ ]:
print("\n--- Cleaning 'lanes' column ---")

def clean_lanes_value(val):
    if pd.isna(val):  # Handles np.nan, pd.NA, None
        return np.nan

    # Convert to string and strip whitespace, as 'lanes' was read with dtype=str
    s_val = str(val).strip()

    if not s_val:  # Handles empty strings
        return np.nan

    # Method 1: Direct conversion to integer (e.g., "2", "3")
    try:
        return int(s_val)
    except ValueError:
        pass  # Not a simple integer string, try next method

    # Method 2: Conversion to float, then floor (e.g., "1.5" -> 1, "2.0" -> 2)
    try:
        float_val = float(s_val)
        return int(np.floor(float_val))
    except ValueError:
        pass  # Not a float string, try next method

    # Method 3: Parse list-like strings (e.g., "['2', '3']", "['4','2','3']")
    if s_val.startswith('[') and s_val.endswith(']'):
        try:
            # ast.literal_eval safely parses string representations of Python literals
            parsed_list = ast.literal_eval(s_val)

            if not isinstance(parsed_list, list): # Ensure it was a list
                return np.nan

            numbers_in_list = []
            for item in parsed_list:
                try:
                    # Convert item to float (handles int, float, or string numbers like '2', '1.5')
                    # Then floor and convert to int
                    num = float(item)
                    numbers_in_list.append(int(np.floor(num)))
                except (ValueError, TypeError):
                    # If an item in the list is not a number (e.g., 'yes', None within list)
                    # Mark the entire original list-string as problematic
                    return np.nan
            
            if numbers_in_list: # If we successfully extracted numbers
                return min(numbers_in_list)
            else:
                # List was empty or all items were non-numeric after parsing
                return np.nan
        except (ValueError, SyntaxError, TypeError):
            # ast.literal_eval failed (e.g., malformed list string like "[1, bad, 2]")
            # or an unexpected type was encountered during item processing
            return np.nan
            
    # If none of the above patterns matched, it's a "weird value"
    return np.nan

# Apply the cleaning function to the 'lanes' column
df['lanes_cleaned'] = df['lanes'].apply(clean_lanes_value)

print("\nValue counts for 'lanes_cleaned' (Top 50):")
print(df['lanes_cleaned'].value_counts(dropna=False).head(50))
print(f"\nData type of 'lanes_cleaned': {df['lanes_cleaned'].dtype}")

# Sanity check: Count how many original NaNs are still NaNs
original_nans = df['lanes'].isna().sum()
cleaned_nans = df['lanes_cleaned'].isna().sum()
print(f"Original NaNs in 'lanes': {original_nans}")
print(f"NaNs in 'lanes_cleaned' (includes original NaNs and newly created NaNs from unparseable values): {cleaned_nans}")

print("-" * 50)

In [ ]:
print("\n--- Cleaning 'highway' column ---")

# Define the hierarchy: Lower index = higher priority
highway_hierarchy_list = [
    'motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'busway',
    'unclassified', 'residential', 'living_street', 'service', 'road',
    'cycleway', 'footway', 'pedestrian', 'path', 'steps', 'track', 'emergency_bay'
]
# Create a mapping from highway type to its priority (index)
highway_priority_map = {road_type: i for i, road_type in enumerate(highway_hierarchy_list)}

def clean_highway_value(val, priority_map):
    if pd.isna(val):
        return np.nan # Will be filled later

    # Convert to string, lowercase, and strip whitespace
    s_val = str(val).lower().strip()

    if not s_val: # Handle empty strings after stripping
        return np.nan

    # Split by comma, then strip whitespace from each part
    parts = [p.strip() for p in s_val.split(',')]

    processed_parts = []
    for part in parts:
        if not part: # Skip empty strings resulting from "a, ,b"
            continue
        # Handle _link: e.g., "secondary_link" -> "secondary"
        if part.endswith("_link"):
            part = part[:-5] # Remove "_link"
        processed_parts.append(part)

    if not processed_parts: # If all parts were empty or just delimiters
        return np.nan

    # Find the highest priority part based on the hierarchy
    # Assign a very high number (low priority) to types not in the hierarchy map
    best_part_found = None
    current_highest_priority = float('inf') # Lower value means higher priority

    for part in processed_parts:
        priority = priority_map.get(part, float('inf')) # Get priority, default to very low if not in map
        if priority < current_highest_priority:
            current_highest_priority = priority
            best_part_found = part
        # If two parts have the same priority (e.g. "primary, primary_link" -> both become "primary"),
        # the first one encountered is kept.

    # If no part was found in the hierarchy map (e.g. "random_tag, another_random"),
    # it will return the first part from processed_parts.
    # If only one part was present and it's not in hierarchy, it's returned.
    if best_part_found is None and processed_parts: # Should only happen if all parts are not in map
        return processed_parts[0] # Default to the first processed part if none are in hierarchy
        
    return best_part_found


# Apply the cleaning function
df['highway_cleaned'] = df['highway'].apply(lambda x: clean_highway_value(x, highway_priority_map))

print("\nValue counts for 'highway_cleaned' (Top 50) before NaN imputation:")
print(df['highway_cleaned'].value_counts(dropna=False).head(50))

# K-Prototypes expects categorical features to be strings, and NaNs can cause issues.
# Impute NaNs with a specific string category like 'unknown_highway'.
df['highway_cleaned'].fillna('unknown_highway', inplace=True)

print("\nValue counts for 'highway_cleaned' (Top 50) after NaN imputation:")
print(df['highway_cleaned'].value_counts(dropna=False).head(50))
print(f"\nData type of 'highway_cleaned': {df['highway_cleaned'].dtype}")
print(f"Number of unique categories in 'highway_cleaned': {df['highway_cleaned'].nunique()}")
print("-" * 50)

In [ ]:
print("\n--- Cleaning 'maxspeed' column ---")

# Conversion factor
MPH_TO_KPH = 1.60934

# Textual speed conversion map (KPH values)
# These will be converted to string later if they are to be categorical.
text_speed_map_kph = {
    'ru:urban': 60,
    'ro:urban': 50,
    'rs:urban': 50,
}

def parse_single_speed_value(speed_token_str):
    """
    Parses a single speed token (e.g., "25 mph", "RU:urban", "50")
    into a numeric KPH value. Returns None if unparseable.
    """
    token = speed_token_str.lower().strip()

    if not token:
        return None

    # 1. Check textual map
    if token in text_speed_map_kph:
        return float(text_speed_map_kph[token])

    # 2. Check for MPH
    if 'mph' in token:
        # Extract numbers (can handle "20 mph" or "20mph")
        numbers = re.findall(r"[-+]?\d*\.\d+|\d+", token)
        if numbers:
            try:
                mph_val = float(numbers[0])
                kph_val = mph_val * MPH_TO_KPH
                # Round to nearest 5 KPH
                return round(kph_val / 5.0) * 5.0
            except ValueError:
                return None # Should not happen if re.findall works
        else: # 'mph' present but no number found
            return None

    # 3. Try direct numeric conversion (assume KPH)
    try:
        # This handles integers "50" and floats "45.5"
        return float(token)
    except ValueError:
        return None # Not a direct number, not mph, not in text map


def clean_maxspeed_value(val):
    if pd.isna(val):
        return np.nan # Keep NaNs

    s_val = str(val).strip()
    if not s_val:
        return np.nan

    # Split by common delimiters like comma or semicolon
    # Allows for "30, 50" or "20 mph; 30"
    parts = re.split(r'[,;]', s_val)

    kph_values_found = []
    for part_str in parts:
        if not part_str.strip(): # Skip empty parts from "val1,,val2"
            continue
        numeric_kph = parse_single_speed_value(part_str)
        if numeric_kph is not None:
            kph_values_found.append(numeric_kph)

    if not kph_values_found: # No valid KPH value could be parsed from any part
        return np.nan

    # Choose the lowest KPH value
    final_kph_value = min(kph_values_found)

    # Convert to integer if it's a whole number (e.g., 50.0 -> 50), then to string
    # This ensures "50.0" becomes "50" for categorical use.
    if final_kph_value == int(final_kph_value):
        return str(int(final_kph_value))
    else:
        # For speeds like 42.5 (if rounding to nearest 2.5 was used, or came from data)
        # Keep one decimal place or as is for the string representation.
        # For rounding to nearest 5, this case (non-integer) should be rare unless original data had it.
        return str(final_kph_value)


# Apply the cleaning function
df['maxspeed_cleaned'] = df['maxspeed'].apply(clean_maxspeed_value)

print("\nValue counts for 'maxspeed_cleaned' (Top 50) before any further NaN handling:")
print(df['maxspeed_cleaned'].value_counts(dropna=False).head(50))

print(f"\nData type of 'maxspeed_cleaned': {df['maxspeed_cleaned'].dtype}")
print(f"Number of unique categories in 'maxspeed_cleaned' (excl NaN): {df['maxspeed_cleaned'].nunique()}")
print("-" * 50)

In [ ]:
print("\n--- Handling Numerical Features: Imputation and Outlier Treatment ---")

# --- 1. Define Numerical Columns to Process ---
# These are the raw numerical columns and the newly cleaned 'lanes_cleaned'
# We process source columns for TotalCrossingCount *before* summing.
numerical_cols_to_process = [
    'lanes_cleaned',                 # Already created and is float (can have NaN)
    'traffic_sign_count',            # Original numerical
    'highway_traffic_signals_count', # Original numerical
    'crossing_traffic_signals_count',# Original numerical (source for TotalCrossingCount)
    'crossing_unmarked_count',       # Original numerical (source for TotalCrossingCount)
    'crossing_uncontrolled_count',   # Original numerical (source for TotalCrossingCount)
    'highway_stop_count',            # Original numerical
    'highway_give_way_count'         # Original numerical
]

# Ensure these columns exist in the DataFrame
numerical_cols_to_process = [col for col in numerical_cols_to_process if col in df.columns]
if not numerical_cols_to_process:
    print("No numerical columns found to process. Skipping this section.")
else:
    print(f"Numerical columns to process: {numerical_cols_to_process}")

    # --- 2. Impute NaNs in Numerical Columns ---
    print("\n--- Imputing NaNs in Numerical Columns ---")
    for col in numerical_cols_to_process:
        if df[col].isnull().any():
            original_nan_count = df[col].isnull().sum()
            if col == 'lanes_cleaned':
                # For 'lanes_cleaned', median imputation is often a good choice.
                # Or a specific default like 1 or 2 if that makes more sense.
                impute_value = df[col].median()
                # Ensure impute_value is not NaN itself if the column is all NaNs
                if pd.isna(impute_value):
                    impute_value = 1 # Fallback if median is NaN
                print(f"Imputing NaNs in '{col}' with median: {impute_value} (Original NaNs: {original_nan_count})")
            else: # For count columns, 0 is a common and sensible imputation
                impute_value = 0
                print(f"Imputing NaNs in '{col}' with 0 (Original NaNs: {original_nan_count})")
            df[col].fillna(impute_value, inplace=True)
        else:
            print(f"No NaNs found in '{col}'.")
    print("-" * 30)

    # --- 3. Visualize Distributions and Identify Outliers ---
    print("\n--- Visualizing Numerical Distributions (before outlier treatment) ---")
    for col in numerical_cols_to_process:
        if col not in df.columns:
            print(f"Column {col} not found for visualization, skipping.")
            continue
        
        print(f"\n--- {col} ---")
        print(df[col].describe())

        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        sns.histplot(df[col], kde=True, bins=50)
        plt.title(f'Histogram of {col}')

        plt.subplot(1, 2, 2)
        sns.boxplot(x=df[col])
        plt.title(f'Boxplot of {col}')

        plt.tight_layout()
        plt.show()
    print("-" * 30)

    # --- 4. Apply Outlier Treatment (Capping/Winsorization) ---
    print("\n--- Applying Outlier Treatment (Capping) ---")
    # Define capping percentiles.
    capping_percentiles = {
        'lanes_cleaned': 0.995,
        'traffic_sign_count': 0.995,
        'highway_traffic_signals_count': 0.995,
        'crossing_traffic_signals_count': 0.995,
        'crossing_unmarked_count': 0.995,
        'crossing_uncontrolled_count': 0.995,
        'highway_stop_count': 0.995,
        'highway_give_way_count': 0.995
    }

    for col in numerical_cols_to_process:
        if col not in df.columns:
            print(f"Column {col} not found for capping, skipping.")
            continue

        # Upper Capping
        percentile_val = capping_percentiles.get(col, 0.995)
        upper_limit = df[col].quantile(percentile_val)

        # Ensure upper_limit is not NaN (can happen if column is all NaNs or has very few distinct values)
        if pd.isna(upper_limit):
            print(f"Could not determine upper limit for '{col}' (quantile is NaN). Skipping capping for this column.")
            continue
            
        original_max = df[col].max()
        
        # Apply capping: values > upper_limit become upper_limit
        df[col] = np.where(df[col] > upper_limit, upper_limit, df[col])

        df[col] = df[col].round().astype(int) # Round to the nearest whole number and convert back to integer
        
        capped_count = (df[col] == upper_limit).sum() # Count how many were set to the upper limit
        
        print(f"Capped '{col}' at {percentile_val*100:.1f}th percentile ({upper_limit:.2f}). "
              f"Original max: {original_max:.2f}. Values at new max: {capped_count}.")

    print("-" * 30)

    # --- 5. Visualize Distributions (after outlier treatment) ---
    print("\n--- Visualizing Numerical Distributions (after outlier treatment) ---")
    for col in numerical_cols_to_process:
        if col not in df.columns:
            continue # Already printed message if not found
            
        print(f"\n--- {col} (After Capping) ---")
        print(df[col].describe())

        plt.figure(figsize=(12, 4))

        plt.subplot(1, 2, 1)
        sns.histplot(df[col], kde=True, bins=50)
        plt.title(f'Histogram of {col} (Capped)')

        plt.subplot(1, 2, 2)
        sns.boxplot(x=df[col])
        plt.title(f'Boxplot of {col} (Capped)')

        plt.tight_layout()
        plt.show()
    print("-" * 50)

In [ ]:
print("\n--- Finalizing Cleaned Columns and Reverting to Original Names ---")

# --- 1. Handle columns that had a '_cleaned' version ---

# For 'lanes'
if 'lanes_cleaned' in df.columns:
    if 'lanes' in df.columns: # If original 'lanes' column still exists
        df.drop(columns=['lanes'], inplace=True) # Drop original raw 'lanes'
    df.rename(columns={'lanes_cleaned': 'lanes'}, inplace=True) # Rename 'lanes_cleaned' to 'lanes'
    print("Finalized 'lanes' column from 'lanes_cleaned'.")
else:
    print("Warning: 'lanes_cleaned' not found. 'lanes' column might not be fully processed as intended.")

# For 'highway'
if 'highway_cleaned' in df.columns:
    if 'highway' in df.columns:
        df.drop(columns=['highway'], inplace=True)
    df.rename(columns={'highway_cleaned': 'highway'}, inplace=True)
    print("Finalized 'highway' column from 'highway_cleaned'.")
else:
    print("Warning: 'highway_cleaned' not found. 'highway' column might not be fully processed as intended.")

# For 'maxspeed'
if 'maxspeed_cleaned' in df.columns:
    if 'maxspeed' in df.columns:
        df.drop(columns=['maxspeed'], inplace=True)
    df.rename(columns={'maxspeed_cleaned': 'maxspeed'}, inplace=True)
    print("Finalized 'maxspeed' column from 'maxspeed_cleaned'.")
else:
    print("Warning: 'maxspeed_cleaned' not found. 'maxspeed' column might not be fully processed as intended.")

# --- 2. Verify other directly cleaned columns (if they were not renamed but cleaned in place) ---
# These columns are assumed to have been cleaned in place (NaNs, outliers for numericals; NaNs, standardization for categoricals)
# No renaming needed for these if cleaned in place. Just a good place for a final check/print.

numerical_features_processed_inplace = [
    'traffic_sign_count',
    'highway_traffic_signals_count',
    'crossing_traffic_signals_count',
    'crossing_unmarked_count',
    'crossing_uncontrolled_count',
    'highway_stop_count',
    'highway_give_way_count'
]
print("\nVerifying numerical features processed in place:")
for col in numerical_features_processed_inplace:
    if col in df.columns:
        print(f"  '{col}': NaNs = {df[col].isnull().sum()}, dtype = {df[col].dtype}")
    else:
        print(f"  Warning: Expected numerical column '{col}' not found.")

categorical_features_processed_inplace = [
    'cycle_lane',
    'shared_cycle',
    'bus_lane'
]
print("\nVerifying categorical features processed in place (for HasSpecialLane):")
for col in categorical_features_processed_inplace:
    if col in df.columns:
        print(f"  '{col}': NaNs = {df[col].isnull().sum()}, dtype = {df[col].dtype}, "
              f"Unique values (sample): {df[col].unique()[:5]}") # Show some unique values
    else:
        print(f"  Warning: Expected categorical column '{col}' not found.")

print("-" * 50)

# Preparing data for clustering

In [ ]:
# 1. Create 'HasSpecialLane' (Binary Categorical)
# Define what values indicate presence of a special lane
cycle_lane_presence = ['yes']
shared_cycle_presence = ['yes']
bus_lane_presence = ['yes']

df['HasSpecialLane'] = (
    df['cycle_lane'].isin(cycle_lane_presence) |
    df['shared_cycle'].isin(shared_cycle_presence) |
    df['bus_lane'].isin(bus_lane_presence)
).astype(int) # Convert True/False to 1/0

print("Created 'HasSpecialLane'. Sample values:")
print(df[['cycle_lane', 'shared_cycle', 'bus_lane', 'HasSpecialLane']].head())

In [ ]:
# 2. Create 'TotalCrossingCount' (Numerical)
crossing_cols_to_sum = [
    'crossing_traffic_signals_count',
    'crossing_unmarked_count',
    'crossing_uncontrolled_count'
]
# Ensure columns exist and are numeric. Handle potential NaNs if necessary (sum defaults to skipna=True)
df['TotalCrossingCount'] = df[crossing_cols_to_sum].sum(axis=1)

print("\nCreated 'TotalCrossingCount'. Sample values:")
print(df[crossing_cols_to_sum + ['TotalCrossingCount']].head())

In [ ]:
# 3. Define the final list of features to keep
final_feature_list = [
    'traffic_sign_count',             # Numerical
    'maxspeed',                       # Categorical
    'lanes',                          # Numerical
    'highway',                        # Categorical
    'highway_traffic_signals_count',  # Numerical
    'HasSpecialLane',                 # Categorical (Binary 0/1)
    'TotalCrossingCount'              # Numerical
]

In [ ]:
# 4. Create the final DataFrame for clustering
df_cluster = df[final_feature_list].copy()

print(f"\nCreated final DataFrame 'df_cluster ' with {df_cluster.shape[1]} features.")
print("Columns:", df_cluster.columns.tolist())
print("Shape:", df_cluster.shape)
print("First 5 rows of selected data:")
print(df_cluster.head())

In [ ]:
# 5. Identify Numerical and Categorical Columns ---
numerical_cols = ['lanes', 'traffic_sign_count', 'highway_traffic_signals_count', 'TotalCrossingCount']
categorical_cols = ['maxspeed', 'highway', 'HasSpecialLane']

for col in categorical_cols:
    df_cluster[col] = df_cluster[col].astype(str)

# 6. Standardize numerical features (robustly handle missing columns)
numerical_cols = [col for col in df_cluster if col not in categorical_cols]
# Keep only existing numerical features
numerical_cols = [num_col for num_col in numerical_cols if num_col in df_cluster.columns]

if numerical_cols:  # Only standardize if there are numerical features
    scaler = StandardScaler()
    df_cluster[numerical_cols] = scaler.fit_transform(df_cluster[numerical_cols])

# Define categorical_indices before using it in fit_predict
categorical_indices = [df_cluster.columns.get_loc(col) for col in categorical_cols if col in df_cluster.columns]  # Get indices of categorical columns

print(f"\nCategorical feature indices for k-Prototypes: {categorical_indices}")
print(f"Numerical features: {numerical_cols}")
print(f"Categorical features: {categorical_cols}")

In [ ]:
# 7. Calculate Variability/Informativeness Scores ---
variability_scores = {}

# Numerical Variables: Variance
print("\n--- Calculating Raw Variance for Numerical Features ---")
variances = df[numerical_cols].var()
print("Raw Variances:")
print(variances)
print("-" * 30)
variability_scores.update(variances.to_dict())

# Categorical Variables: Entropy
print("\n--- Calculating Raw Entropy for Categorical Features ---")
raw_entropies = {}
for col in categorical_cols:
    if df[col].nunique() <= 1:
        # Handle columns with only one category (zero entropy)
        score = 0.0
    else:
        # Calculate probability distribution (frequencies)
        p_data = df[col].value_counts(normalize=True)
        # Calculate entropy
        score = calculate_entropy(p_data)
    variability_scores[col] = score
    raw_entropies[col] = score # Store raw entropy too for display

print("Raw Entropies:")
print(pd.Series(raw_entropies))
print("-" * 30)

# Create a DataFrame for scores
scores_df = pd.DataFrame.from_dict(variability_scores, orient='index', columns=['Score'])

In [ ]:
# 8. Normalize Scores ---
# Use MinMaxScaler to scale scores to [0, 1]
# Reshape scores for the scaler (it expects a 2D array)
scores = scores_df['Score'].values.reshape(-1, 1)

# Handle potential case where all scores are the same (avoid division by zero)
if np.all(scores == scores[0]):
     print("Warning: All variability/entropy scores are identical. Normalization might not be meaningful.")
     # Assign a uniform normalized score (e.g., 0.5) or handle as needed
     normalized_scores = np.full_like(scores, 0.5)
else:
    scaler = MinMaxScaler()
    normalized_scores = scaler.fit_transform(scores)

scores_df['NormalizedScore'] = normalized_scores
scores_df = scores_df.sort_values(by='NormalizedScore', ascending=False)

print("Variability/Informativeness Scores (Normalized):")
print(scores_df)

In [ ]:
# 9. Qualitative Analysis & Selection ---
# Note: The following selection methods are for analytical purposes and do not alter df_cluster.
# Option A: Threshold
threshold = 0.2 # Example threshold
selected_features_thresh = scores_df[scores_df['NormalizedScore'] > threshold].index.tolist()
print(f"\nFeatures that should be selected using threshold > {threshold}:", selected_features_thresh)

# Option B: Top N
top_n = 8 # Example: Select top 8 features
selected_features_topn = scores_df.head(top_n).index.tolist()
print(f"\nFeatures that should be selected if picking Top {top_n}:", selected_features_topn)

# Option C: Manual Selection based on plot/domain knowledge
scores_df['NormalizedScore'].plot(kind='bar', figsize=(10, 5))
plt.title('Normalized Variability/Informativeness Scores')
plt.ylabel('Normalized Score')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# 10. Check for Redundancy Among Features ---

print("\nChecking Redundancy...")
# Numerical Correlation
if len(numerical_cols) > 1:
    corr_matrix = df[numerical_cols].corr()
    print("\nCorrelation Matrix (Selected Numerical):")
    print(corr_matrix)
    # Visualize heatmap
    plt.figure(figsize=(8, 6))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Correlation Matrix of Selected Numerical Features')
    plt.show()
    # Decision: If high correlation (e.g., > |0.8|) is found between two variables,
    # consider removing one based on domain knowledge or lower variability score.

# Categorical Association (using Cramer's V)
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    if min((kcorr-1), (rcorr-1)) == 0:
         # Handle cases with zero degrees of freedom (e.g., constant variable)
         # This shouldn't happen if we selected based on non-zero entropy/variance,
         # but good to have a fallback.
         return 0.0
    else:
         return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))


if len(categorical_cols) > 1:
    print("\nCramer's V (Selected Categorical):")
    cramer_results = pd.DataFrame(index=categorical_cols, columns=categorical_cols)
    for col1 in categorical_cols:
        for col2 in categorical_cols:
            if col1 == col2:
                cramer_results.loc[col1, col2] = 1.0
            else:
                cramer_results.loc[col1, col2] = cramers_v(df[col1], df[col2])
    # Convert to numeric after filling
    cramer_results = cramer_results.astype(float)
    print(cramer_results)
    # Visualize
    plt.figure(figsize=(8, 6))
    sns.heatmap(cramer_results, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title("Cramer's V for Selected Categorical Features")
    plt.show()
    # Decision: If high Cramer's V (e.g., > 0.7) is found, consider removing one.

# DETERMINING OPTIMAL K, USING 2 METHODS

## Determining optimal k using the Elbow Method

In [ ]:
costs = []
K = range(1, 11)  # Test cluster numbers from 1 to 10 (adjust range as needed)

for num_clusters in list(K): #iterate through cluster numbers
    print(f"Running Cluster {num_clusters}:")
    kproto = KPrototypes(n_clusters=num_clusters, init='Cao', verbose=2, random_state=0, n_jobs=-1) #n_jobs=-1 uses all available cores
    kproto.fit_predict(df_cluster, categorical=categorical_indices)
    costs.append(kproto.cost_) #Append cost to costs list

plt.plot(K, costs, 'bx-')
plt.xlabel('Number of Clusters')
plt.ylabel('Cost')
plt.savefig(os.path.join(FIGURES_DIR, 'elbow_method.png'))
plt.show() #Plot elbow

## Determining Optimal k using the Silhouette Method

In [ ]:
print("Starting the Silhouette Method")
silhouette_avg = []
K = range(2, 11)  # Adjust range as needed

for num_clusters in list(K):
    print(f"Running Cluster {num_clusters}:")
    kproto = KPrototypes(n_clusters=num_clusters, init='Cao', verbose=2, random_state=0, n_jobs=-1)
    cluster_labels = kproto.fit_predict(df_cluster, categorical=categorical_indices)

    # --- One-hot encode categorical features before calculating Silhouette score ---
    df_silhouette = df_cluster.copy()  # Create a copy for one-hot encoding
    df_silhouette = pd.get_dummies(df_silhouette, columns=categorical_cols)
    silhouette_avg = silhouette_score(df_silhouette.values, cluster_labels)  # Average score for all samples

    sample_silhouette_values = silhouette_samples(df_silhouette.values, cluster_labels) #Silhouette values for each sample

    # --- Create the Silhouette plot ---
    n_clusters = num_clusters  # Assign num_clusters to n_clusters so it is defined in this scope.
    fig, ax1 = plt.subplots(1, 1)
    fig.set_size_inches(7, 5)
    ax1.set_xlim([-0.1, 1])  #xlim and ylim values are fixed, adapt if needed
    ax1.set_ylim([0, len(df_silhouette) + (n_clusters + 1) * 10]) # adapt if needed
    y_lower = 10

    for i in range(n_clusters): #Iterate through cluster labels

        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i] #Silhouette scores for current cluster
        ith_cluster_silhouette_values.sort() #Sort silhouette scores

        size_cluster_i = ith_cluster_silhouette_values.shape[0] #number of samples in cluster i
        y_upper = y_lower + size_cluster_i


        color = plt.cm.nipy_spectral(float(i) / n_clusters)  # Customize colors if needed
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i)) #Label the silhouette plots with their cluster numbers at the middle


        y_lower = y_upper + 10  # 10 for the 0 samples


    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])



    plt.suptitle(("Silhouette analysis for KPrototypes clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')
    plt.savefig(os.path.join(FIGURES_DIR, f'silhouette_full_data_k_{n_clusters}.png'))
    plt.show()

## Determining Optimal k using the Silhouette Method on a Sample

In [ ]:
print("Starting the Silhouette Method using Sampling")
silhouette_scores_avg = [] # Store average scores for each k
K = range(3, 6)  # Adjust range as needed
SAMPLE_SIZE = 700000  # Or 5000, 20000 etc. Adjust based on your RAM. Needs to be large enough to be representative.
if SAMPLE_SIZE > len(df_cluster):
    SAMPLE_SIZE = len(df_cluster) # Don't sample more than available data

for num_clusters in K:
    print(f"\nRunning KPrototypes for k = {num_clusters} on full data...")
    kproto = KPrototypes(n_clusters=num_clusters, init='Cao', verbose=2, random_state=0, n_jobs=-1) # Use all cores

    # Fit on FULL data, predict on FULL data
    cluster_labels = kproto.fit_predict(df_cluster.values, categorical=categorical_indices)
    print(f"KPrototypes done for k = {num_clusters}.")

    # --- Perform Sampling for Silhouette ---
    print(f"Sampling {SAMPLE_SIZE} points for Silhouette calculation...")
    if len(df_cluster) > SAMPLE_SIZE:
        # Create random indices
        sample_indices = np.random.choice(df_cluster.index, SAMPLE_SIZE, replace=False)
        # Select sample data AND corresponding labels
        df_sample = df_cluster.loc[sample_indices]
        labels_sample = cluster_labels[sample_indices]
    else:
        # If dataset is smaller than sample size, use all data
        df_sample = df_cluster
        labels_sample = cluster_labels

    print("One-hot encoding the sample...")
    # One-hot encode ONLY the sample
    df_silhouette_sample = pd.get_dummies(df_sample, columns=categorical_cols)

    print("Calculating Silhouette score on the sample...")
    # Calculate Silhouette score using ONLY the sample data and labels
    silhouette_avg = silhouette_score(df_silhouette_sample.values, labels_sample)  # Average score for all samples
    silhouette_scores_avg.append(silhouette_avg)
    print(f"Average Silhouette Score for k = {num_clusters}: {silhouette_avg:.4f}")
    sample_silhouette_values = silhouette_samples(df_silhouette_sample.values, labels_sample) #Silhouette values for each sample
      
    # --- Create the Silhouette plot ---
    print("Start plotting...")
    n_clusters = num_clusters  # Assign num_clusters to n_clusters so it is defined in this scope.
    fig, ax1 = plt.subplots(1, 1)
    fig.set_size_inches(7, 5)
    ax1.set_xlim([-0.1, 1])  #xlim and ylim values are fixed, adapt if needed
    ax1.set_ylim([0, len(df_silhouette_sample) + (n_clusters + 1) * 10]) # adapt if needed
    y_lower = 10

    for i in range(n_clusters): #Iterate through cluster labels

        ith_cluster_silhouette_values = sample_silhouette_values[labels_sample == i] #Silhouette scores for current cluster
        ith_cluster_silhouette_values.sort() #Sort silhouette scores

        size_cluster_i = ith_cluster_silhouette_values.shape[0] #number of samples in cluster i
        y_upper = y_lower + size_cluster_i


        color = plt.cm.nipy_spectral(float(i) / n_clusters)  # Customize colors if needed
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i)) #Label the silhouette plots with their cluster numbers at the middle


        y_lower = y_upper + 10  # 10 for the 0 samples


    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])



    plt.suptitle(("Silhouette analysis for KPrototypes clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')
    plt.savefig(os.path.join(FIGURES_DIR, f'silhouette_sampled_k_{n_clusters}.png'))
    plt.close(fig) # Close the figure to free up memory

    # --- Optional: Clean up large objects ---
    del df_sample, labels_sample, df_silhouette_sample, kproto, cluster_labels
    gc.collect() # Force garbage collection (might help, might not)

# --- Plotting the results (handling potential NaNs) ---
valid_indices = ~np.isnan(silhouette_scores_avg) # Find where scores are not NaN
valid_K = np.array(list(K))[valid_indices]
valid_scores = np.array(silhouette_scores_avg)[valid_indices]

# --- Plotting the results (Average Silhouette Scores vs K) ---
plt.figure(figsize=(8, 5))
plt.plot(K, silhouette_scores_avg, 'bo-')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Average Silhouette Score (Sampled)')
plt.title('Silhouette Score for Determining Optimal k (using Sampling)')
plt.grid(True)
plt.xticks(list(K))
plt.savefig(os.path.join(FIGURES_DIR, 'silhouette_method_average_scores_sampled.png'))
plt.show()

# K-Prototypes clustering

In [ ]:
# --- Choose optimal k based on the elbow plot ---
# Examine the Elbow and Silhouette performed before and select the k value at the "elbow" point
# (where the rate of decrease in cost starts to slow down significantly).
#Then, update below the value of n_clusters

n_clusters=4

# --- Perform K-Prototypes clustering with the optimal k ---
print(f"Running K-Prototypes with {n_clusters} clusters.")
kproto = KPrototypes(n_clusters=n_clusters, init='Cao', verbose=2, random_state=0, n_jobs=-1)
clusters = kproto.fit_predict(df_cluster, categorical=categorical_indices)

df['cluster'] = clusters

# --- Analyze cluster characteristics (with debugging prints) ---
print(df['cluster'].value_counts())  # Number of edges in each cluster

for i in range(n_clusters):
    cluster_data = df[df['cluster'] == i]
    print(f"\nCluster {i}:")
    #print(f"cluster_data.dtypes:\n{cluster_data.dtypes}") #Print data types
    #print(f"cluster_data head:\n{cluster_data.head()}") # Print head of cluster_data

    if cluster_data.empty: #check if empty and skip if so
        print(f"cluster_data is empty. Skipping cluster {i}")
        continue

    for feature in final_feature_list:
        
        if cluster_data[feature].apply(type).eq(list).any():
          print(f"  {feature}:")
          exploded_counts = cluster_data.explode(feature)[feature].value_counts().to_dict()
          for val, count in exploded_counts.items():
            print(f"    {val}: {count}")    
        else:  # Proceed as usual if no lists found
            print(f"  {feature}: {cluster_data[feature].value_counts().to_dict()}")

print("Clustering complete.")

# Saving final results

In [ ]:
cluster_results_gpkg = os.path.join(RESULTS_DIR, "full_network_with_clusters.gpkg")
cluster_results_csv = os.path.join(RESULTS_DIR, "full_network_with_clusters.csv")

# Drop city_id before final save
df_to_save = df.drop(columns=['city_id'], errors='ignore')

# --- 1. Save to GeoPackage (GPKG) ---
try:
    print(f"Saving clustered network with geometry to: {cluster_results_gpkg}")
    df_to_save.to_file(cluster_results_gpkg, driver="GPKG")
except Exception as e_save_gpkg:
    print(f"Error saving clustered GPKG: {e_save_gpkg}")

# --- 2. Save to Comma-Separated Values (CSV) ---
try:
    print(f"Saving clustered attributes to: {cluster_results_csv}")
    df_to_save.to_csv(cluster_results_csv, sep=";", index=False)
except Exception as e_csv:
    print(f"Error saving clustered CSV: {e_csv}")